In [1]:
# code from https://medium.com/district-data-labs/building-a-classifier-from-census-data-18f996c4d7cf
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss

import pandas as pd

# prepare train and test data
names = [
    'age',
    'workclass',
    'fnlwgt',
    'education',
    'education-num',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital-gain',
    'capital-loss',
    'hours-per-week',
    'native-country',
    'income',
]
data = pd.read_csv('census_data/adult.data', names=names)

import json
meta = {
    'target_names': list(data.income.unique()),
    'feature_names': list(data.columns),
    'categorical_features': {
        column: list(data[column].unique())
        for column in data.columns
        if data[column].dtype == 'object'
    },
}
with open('census_data/meta.json', 'w') as f:
    json.dump(meta, f, indent=2)
    
from sklearn.datasets.base import Bunch
from sklearn.preprocessing import LabelEncoder
def load_data():
    # Load the meta data from the file
    with open('census_data/meta.json', 'r') as f:
        meta = json.load(f)
    names = meta['feature_names']
    # Load the training and test data, skipping the bad row in the test data
    train = pd.read_csv('census_data/adult.data', names=names)
    test  = pd.read_csv('census_data/adult.test', names=names, skiprows=1)
    # Remove the target from the categorical features
    meta['categorical_features'].pop('income')
    # Return the bunch with the appropriate data chunked apart
    return Bunch(
        data = train[names[:-1]],
        target = train[names[-1]],
        data_test = test[names[:-1]],
        target_test = test[names[-1]],
        target_names = meta['target_names'],
        feature_names = meta['feature_names'],
        categorical_features = meta['categorical_features'],
        DESCR = "descr",
    )
dataset = load_data()

from sklearn.base import BaseEstimator, TransformerMixin
class EncodeCategorical(BaseEstimator, TransformerMixin):
    """
    Encodes a specified list of columns or all columns if None.
    """
    def __init__(self, columns=None):
        self.columns  = columns
        self.encoders = None
    def fit(self, data, target=None):
        """
        Expects a data frame with named columns to encode.
        """
        # Encode all columns if columns is None
        if self.columns is None:
            self.columns = data.columns
        # Fit a label encoder for each column in the data frame
        self.encoders = {
            column: LabelEncoder().fit(data[column])
            for column in self.columns
        }
        return self
    def transform(self, data):
        """
        Uses the encoders to transform a data frame.
        """
        output = data.copy()
        for column, encoder in self.encoders.items():
            output[column] = encoder.transform(data[column])
        return output
encoder = EncodeCategorical(dataset.categorical_features.keys())
dataset.data = encoder.fit_transform(dataset.data)
dataset.data_test = encoder.fit_transform(dataset.data_test)

from sklearn.impute import SimpleImputer
import numpy as np

class ImputeCategorical(BaseEstimator, TransformerMixin):
    """
    Encodes a specified list of columns or all columns if None.
    """
    def __init__(self, columns=None):
        self.columns = columns
        self.imputer = None
    def fit(self, data, target=None):
        """
        Expects a data frame with named columns to impute.
        """
        # Encode all columns if columns is None
        if self.columns is None:
            self.columns = data.columns
        # Fit an imputer for each column in the data frame
        self.imputer = SimpleImputer(missing_values=0, strategy='most_frequent')
        self.imputer.fit(data[self.columns])
        return self
    def transform(self, data):
        """
        Uses the encoders to transform a data frame.
        """
        output = data.copy()
        output[self.columns] = self.imputer.transform(output[self.columns])
        return output
imputer = ImputeCategorical(['workclass', 'native-country', 'occupation'])
dataset.data = imputer.fit_transform(dataset.data)
dataset.data_test = imputer.fit_transform(dataset.data_test)

X_train = dataset.data
yencode = LabelEncoder().fit(dataset.target)
y_train = yencode.transform(dataset.target)

X_test = dataset.data_test
y_test = yencode.transform([y.rstrip(".") for y in dataset.target_test])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.datasets.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
from davincicode import DaVinciCode
app = DaVinciCode(port=8090, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
~/Documents/GitHub/ml-viz/ml-viz/davincicode/DaVinciCode.py in update_pc(
    clickData='main/MLPClassifier/max_iter=nan/',
    rangeData=[0, 1],
    n=242
)
    428                     else:
    429                         selected_df = selected_df[selected_df['model'] == i]
--> 430                 sample_vals = selected_df.iloc[0]
        sample_vals = undefined
        selected_df.iloc = <pandas.core.indexing._iLocIndexer object at 0x120aacb38>
    431                 labels_pc = {}
    432                 for i in self.hierarchy_path[subset_counter:]:

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py in __getitem__(
    self=<pandas.core.indexing._iLocIndexer object>,
    key=0
)
   1766 
   1767             maybe_callable = com.apply_if_callable(key, self.obj)
-> 1768            

In [3]:
from sklearn.neural_network import MLPClassifier
app.experiment(library='sklearn', model=MLPClassifier, params = {
    'max_iter': 100,
    'alpha': 0.001
})

In [ ]:
app.experiment(library='sklearn', model=MLPClassifier, params = {
    'max_iter': 400,
    'alpha': 0.0001
})

app.experiment(library='sklearn', model=MLPClassifier, params = {
    'max_iter': 400,
    'alpha': 0.00001
})

app.experiment(library='sklearn', model=MLPClassifier, params = {
    'max_iter': 200,
    'alpha': 0.001
})

In [ ]:
params = {
        'objective': 'multi:softprob',
        'num_class': 2,
        'eval_metric': 'mlogloss',
        'max_depth': 10,
        'min_child_weight': 10,
        'eta':0.2,
        'subsample': 0.05,
        'colsample_bytree': 0.05
}

app.experiment(library='xgboost', model=None, params = params)

In [ ]:
params['max_depth'] = 1
params['min_child_weight'] = 6
params['subsample'] = 1.0
params['colsample_bytree'] = 0.05
params['eta'] = 0.15
app.experiment('xgboost', None, params)

In [ ]:
params['max_depth'] = 10
params['min_child_weight'] = 10
params['subsample'] = 1.0
params['colsample_bytree'] = 1.0
app.experiment('xgboost', None, params)

In [ ]:
from sklearn.svm import SVC
app.experiment('sklearn', SVC, {
    'gamma': 0.01,
    'C': 1
})